<a href="https://colab.research.google.com/github/meiqingli/Geospatial-Software-Design/blob/master/D_Lab_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extracting Time-Series Satellite Images with Google Earth Engine Python API

by: Meiqing Li (meiqing@berkeley.edu), D-Lab Data Science Fellow

This notebook demonstrates the process to filter, classify, resample, and visualiza time-series satellite images using Google Earth Engine Python API. 

### API Setup: import, authenticate and initialize

Beofre running the code, we need to first import the Google Earth Enginer API. Since the Earth Engine API is installed by default in Google Colaboratory, I don't need to install it here. 

In [1]:
import ee

In [2]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=qLJqTz7e1xya_2XxnnXNOYOd19x75uNS4xV65Qj-_dc&tc=w_SKdMI0xNgCwZTKf2FcnZFpO403BepMDE8OqV8KiV0&cc=RRnSUS8gdK-6IK0sK3E1f_mPCqjQPN8PYoZ2z0YUguQ

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AVHEtk7I-T7FosUDVuxuQzUfKBbppR4PKV5rYjiFjlenmrLUVT48pZ5d7YA

Successfully saved authorization token.


### Explore Earth Engine Data Catalog

[The Earth Engine Data Catalog](https://developers.google.com/earth-engine/datasets/catalog) 

https://code.earthengine.google.com/1ec1d414a4c5b03bbb0a7d6c19b5a748
https://code.earthengine.google.com/291961fe8406ca4c75ecea4ef3f33435

VIIRS - Nighttime Light Image


In [ ]:
# Print the elevation of Mount Everest.
dem = ee.Image('USGS/SRTMGL1_003')
xy = ee.Geometry.Point([86.9250, 27.9881])
elev = dem.sample(xy, 30).first().get('elevation').getInfo()
print('Mount Everest elevation (m):', elev)

Mount Everest elevation (m): 8729


### Import Features

In [73]:
# I have uploaded CA county and state boundaries through GEE console (add more description here)
CACounty = ee.FeatureCollection("projects/spherical-depth-278922/assets/California_County_Boundaries")
CAState = ee.FeatureCollection("projects/ee-meiqingli/assets/CAState")

#### DMSP/OLS

https://developers.google.com/earth-engine/datasets/catalog/NOAA_DMSP-OLS_NIGHTTIME_LIGHTS#bands

https://developers.google.com/earth-engine/datasets/catalog/BNU_FGS_CCNL_v1

In [85]:
## DMSP OLS Nighttime Lights (1992-01-01 - 2014-01-01)
# select band and filter date
DMSP = ee.ImageCollection('BNU/FGS/CCNL/v1').select('b1').filterDate('1992-01-01', '2014-01-01')

#### VIIRS

https://developers.google.com/earth-engine/datasets/catalog/NOAA_VIIRS_DNB_MONTHLY_V1_VCMSLCFG#description

In [80]:
## VIIRS Nighttime Lights (2014-01-01 - 2022-11-01)
# select band and filter date
VIIRS = ee.ImageCollection('NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG').select('avg_rad').filterDate('2014-01-01', '2022-01-01')

#### MODIS Land Cover

https://developers.google.com/earth-engine/datasets/catalog/MODIS_061_MCD12Q1

In [131]:
MODIS = ee.ImageCollection('MODIS/006/MOD11A1').select('LST_Day_1km').filterDate('2001-01-01', '2021-01-01')

### Get time-series data for a location

UC Berkeley: [37.8719, -122.2585]

In [156]:
## Calculate the mean value around a location using the `getRegion()` method
# DMSP/OLS
poi = ee.Geometry.Point([-122.3790, 37.6213]) # note lon, lat
scale = 30

DMSP_poi = DMSP.getRegion(poi, scale).getInfo()

# preview the result
DMSP_poi[:5]

[['id', 'longitude', 'latitude', 'time', 'b1'],
 ['DMSP1992v1',
  -122.37897337582122,
  37.621309351632945,
  694224000000,
  67.76784695621606],
 ['DMSP1993v1',
  -122.37897337582122,
  37.621309351632945,
  725846400000,
  56.66487062392774],
 ['DMSP1994v1',
  -122.37897337582122,
  37.621309351632945,
  757382400000,
  81.95031347136096],
 ['DMSP1995v1',
  -122.37897337582122,
  37.621309351632945,
  788918400000,
  91.53101235005826]]

In [157]:
# VIIRS
VIIRS_poi = VIIRS.getRegion(poi, scale).getInfo()

# preview the result
VIIRS_poi[:5]

[['id', 'longitude', 'latitude', 'time', 'avg_rad'],
 ['20140101',
  -122.37897337582122,
  37.621309351632945,
  1388534400000,
  55.463462829589844],
 ['20140201',
  -122.37897337582122,
  37.621309351632945,
  1391212800000,
  72.19701385498047],
 ['20140301',
  -122.37897337582122,
  37.621309351632945,
  1393632000000,
  60.484066009521484],
 ['20140401',
  -122.37897337582122,
  37.621309351632945,
  1396310400000,
  63.16453170776367]]

Transform to pandas dataframe

In [158]:
import pandas as pd

def ee_array_to_df(arr, list_of_bands):
    """Transforms client-side ee.Image.getRegion array to pandas.DataFrame."""
    df = pd.DataFrame(arr)

    # Rearrange the header.
    headers = df.iloc[0]
    df = pd.DataFrame(df.values[1:], columns=headers)

    # Remove rows without data inside.
    df = df[['longitude', 'latitude', 'time', *list_of_bands]].dropna()

    # Convert the data to numeric values.
    for band in list_of_bands:
        df[band] = pd.to_numeric(df[band], errors='coerce')

    # Convert the time field into a datetime.
    df['datetime'] = pd.to_datetime(df['time'], unit='ms')

    # Keep the columns of interest.
    df = df[['time','datetime',  *list_of_bands]]

    return df

In [159]:
dmsp_poi = ee_array_to_df(DMSP_poi,['b1'])
viirs_poi = ee_array_to_df(VIIRS_poi,['avg_rad'])

viirs_poi.head()

,time,datetime,avg_rad
0,1388534400000,2014-01-01,55.463463
1,1391212800000,2014-02-01,72.197014
2,1393632000000,2014-03-01,60.484066
3,1396310400000,2014-04-01,63.164532
4,1398902400000,2014-05-01,52.005966


## Static image

In [114]:
# Import the Image function from the IPython.display module.
from IPython.display import Image

# Display a thumbnail of global nighttime light.
Image(url = VIIRS.mean()
  .getThumbURL({'min': 0, 'max': 63, 'region': CACounty.filterMetadata("COUNTY_NAM","equals", "Alameda").first().geometry(), 'dimensions': 512,
                'palette': ['000044','ffff00','ffffff']}))

## Interactive map

In [77]:
# Import the Folium library.
import folium

# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Set visualization parameters.
vis_params = {
  'min': 0,
  'max': 63,
  'palette': ['000044','ffff00','ffffff']}

# Create a folium map object.
my_map = folium.Map(location=[38, -121], zoom_start=5.5)

# Add the nighttime light to the map object.
my_map.add_ee_layer(NightImage.updateMask(NightImage.gt(0)), vis_params, 'NightImage')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)

### DMSP/OLS Nighttime Lights


### VIIRS Nighttime Light

### References

*   https://colab.research.google.com/github/google/earthengine-api/blob/master/python/examples/ipynb/ee-api-colab-setup.ipynb
*   https://colab.research.google.com/github/google/earthengine-community/blob/master/tutorials/intro-to-python-api/index.ipynb
